The google-cloud-documentai is the official Python client library for Google Cloud Document AI, a service that uses machine learning to automatically extract structured data (like text, form fields, tables, and key-value pairs) from documents such as PDFs, images, and scanned forms.

In [1]:
pip install --upgrade google-cloud-documentai


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# installing google.cloud

In [2]:
pip install google.cloud

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


This installs the Google Cloud Document AI Python client along with all optional dependencies needed for extended functionality.

Includes:
google-cloud-documentai — main client library

proto-plus — for protocol buffer support

google-api-core, grpc — for API communication

google-auth, requests — for authentication and HTTP support

PDF/Image support: Installs additional dependencies for processing local documents.



In [3]:
pip install "google-cloud-documentai[all]"


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Accessing google document ai api 

1. Crreate project
2. create a document ai processor in the project
3. get the json file of project 
4. authenticate and login
5. enable api
install and initialize cloud cli The gcloud CLI provides a set of tools that you can use to manage resources and applications hosted on Google Cloud.

In [ ]:


from google.api_core.client_options import ClientOptions
from google.cloud import documentai_v1

project_id = "wise-env-461717-t5"

processor_id = "86a7eec52bbb9616" 

location = "us"

In [ ]:


# Path for file to process.
file_path =r"C:\Users\AMANPREET KAUR\OneDrive\Desktop\study\Model.pdf"

# Set `api_endpoint` if you use a location other than "us".
opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

# Initialize Document AI client.
client = documentai_v1.DocumentProcessorServiceClient(client_options=opts)

# Get the Fully-qualified Processor path.
full_processor_name = client.processor_path(project_id, location, processor_id)

# Get a Processor reference.
request = documentai_v1.GetProcessorRequest(name=full_processor_name)
processor = client.get_processor(request=request)

# `processor.name` is the full resource name of the processor.
# For example: `projects/{project_id}/locations/{location}/processors/{processor_id}`
print(f"Processor Name: {processor.name}")

# Read the file into memory.
with open(file_path, "rb") as image:
    image_content = image.read()
#Opens the PDF file in binary mode and reads its contents.

# Load binary data.

raw_document = documentai_v1.RawDocument(
    content=image_content,
    mime_type="application/pdf",
)

# Send a request and get the processed document.
request = documentai_v1.ProcessRequest(name=processor.name, raw_document=raw_document)
result = client.process_document(request=request)
document = result.document

# Read the text recognition output from the processor.
# For a full list of `Document` object attributes, reference this page:
# https://cloud.google.com/document-ai/docs/reference/rest/v1/Document
print("The document contains the following text:")
print(document.text)

Processor Name: projects/194546811894/locations/us/processors/86a7eec52bbb9616
The document contains the following text:
System that
Es a collection
act or
accomplishment of
Collection
of
Study
State
untities
interact
might only
to of system
ndative
some
that
be
of
collection
is
describe
a
to
objectives
Systen
entities
together
of people or machines
towards
Logical end.
the
compruse entire system in one
a Subset
of
another.
variables required
particular time
at a
systems
of study.
discrete
State
of
variables
changes
at
of
instantaneously
separated pts
time.
Analytical
State
Continuous
change
of
Wat
variables
continuously
to time.
the
Model
of
to
work
to get
Some
We
solutions US
is
simple
then
with
its
the
Exact
d = rt
can get
Simulation
it may be
relationships
tical
сполу
distance
velocity
time
Solutions
analytical requires
complex analytical
→ inverting
of analytical sol
efficient
vast
possible
and quantities
socuts of
is known
also
+=
2/4.
cau
become
very
computing
large matrix
avail

# Chunk the data create embeddings

In [5]:
from google.cloud import documentai_v1
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import textwrap


# Step 2: Chunk the text
def chunk_text(text, max_chars=500):
    return textwrap.wrap(text, max_chars)

text_chunks = chunk_text(document.text)

# Step 3: Create embeddings for each chunk
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(text_chunks)

# Step 4: Create and populate FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))
faiss.write_index(index, "faiss_index.index")
import pickle

with open("text_chunks.pkl", "wb") as f:
    pickle.dump(text_chunks, f)





AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [6]:
import requests

def ask_groq_agent(query, context):
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    response = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        headers={"Authorization": f"Bearer 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"},
        json={
            "model": "llama3-70b-8192",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.3
        }
    )
    return response.json()["choices"][0]["message"]["content"]


In [7]:
def retrieve_context(query, top_k=5):
    query_embed = embed_model.encode([query])
    distances, indices = index.search(np.array(query_embed), top_k)
    return [text_chunks[i] for i in indices[0]]


In [8]:
def qa_agent(question):
    retrieved_chunks = retrieve_context(question)
    context = "\n\n".join(retrieved_chunks)
    return ask_groq_agent(question, context)


In [9]:
question = "What is the system"
answer = qa_agent(question)
print(answer)


Based on the provided context, a system can be defined as:

"A collection of entities, including people or machines, that interact together towards a logical end, comprising a set of variables that change over time, either continuously or discretely, and can be modeled using analytical, simulation, or static approaches to understand its behavior and evolution."


import os
GROQ_API_KEY = os.getenv("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")


In [10]:
import os
GROQ_API_KEY = os.getenv("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")
